In [1]:
%load_ext autoreload

%autoreload 2

%matplotlib inline

In [4]:
import pandas as pd

from library import *

In [5]:

host = '10.0.5.225'
db = 'billin'
user = 'postgres'
password = 'ppoSgre123'

db = DB(user=user, password=password, host=host, db=db)

In [18]:
paymets_methods = db.gettable('f_invoice_payments')
paymets_methods.set_index('id_invoices', inplace=True)

In [19]:
paymets_methods.provider.unique()
paymets_methods.state.unique()

array(['gocardless', 'stripe', 'unnax'], dtype=object)

In [22]:
paymets_methods.head()

,uuid_payment_id,payment_created_at,payment_updated_at,amount,provider,state,charge_id,transfer_id,sincro_at,id_business
id_invoices,,,,,,,,,,
59f7546e3d496965e9c18b66,5a47562e6fb891690fe2b94f,20171230,20180109,59.29,gocardless,paid_out,PM001M4TEWBJHR,59f7546e3d496965e9c18b66,2018-07-12 21:54:52.567036+00:00,-1
59f7568e388b47657851c58b,5a4756406fb891690fe2b952,20171230,20180109,36.30,gocardless,paid_out,PM001M4TEYR5FF,59f7568e388b47657851c58b,2018-07-12 21:54:52.567036+00:00,53300
5a6cd1f039d60273ecf04f42,5a6cd31186e2c2373c1d3526,20180127,20180127,3.00,stripe,transfer,ch_1BozM5JahFDQqyZkgYN5WvuW,tr_1BozM7JahFDQqyZke7r45WMA,2018-07-12 21:54:52.567036+00:00,53305
5a6ccf6de4c1f8750a151f40,5a6cd496e4c1f8750a151f4f,20180127,20180127,3.00,stripe,transfer,ch_1BozSMJahFDQqyZkHjEedee9,tr_1BozSPJahFDQqyZk5MHZ9jnH,2018-07-12 21:54:52.567036+00:00,53305
5a6ce3e3e4c1f8750a151f58,5a6ce4b927fdb473f149f41b,20180127,20180127,3.00,stripe,succeeded,ch_1Bp0X0JahFDQqyZk2S0smHcd,tr_1Bp0X2JahFDQqyZkWgvnmtvQ,2018-07-12 21:54:52.567036+00:00,53305


In [23]:
provider = pd.get_dummies(paymets_methods.provider)

In [26]:
state = pd.get_dummies(paymets_methods.state)

In [36]:
paymets_methods.payment_updated_at = pd.to_datetime(paymets_methods.payment_updated_at, format='%Y%m%d')
paymets_methods.payment_created_at = pd.to_datetime(paymets_methods.payment_created_at, format='%Y%m%d')

In [38]:
payment_delay = paymets_methods.payment_updated_at - paymets_methods.payment_created_at

In [146]:
amount_dict = dict()
for b in paymets_methods.id_business.unique().tolist():
    amount_dict[b] = {}
    tempdf = paymets_methods[paymets_methods.id_business == b]
    amount_dict[b]['mean'] = tempdf.amount.mean()
    amount_dict[b]['std'] = np.nan_to_num(tempdf.amount.std())
    amount_dict[b]['max'] = np.nan_to_num(tempdf.amount.max())
    amount_dict[b]['min'] = np.nan_to_num(tempdf.amount.min())
    amount_dict[b]['count'] = tempdf.amount.count()
    
    payment_delay = tempdf.payment_updated_at - tempdf.payment_created_at
    amount_dict[b]['payment_delay_mean'] = payment_delay.mean().floor('1d')
    amount_dict[b]['payment_delay_std'] = payment_delay.std().floor('1d')
    amount_dict[b]['payment_delay_max'] = payment_delay.max()
    amount_dict[b]['payment_delay_min'] = payment_delay.min()
    
    for p in tempdf.provider.unique().tolist():
        temp_p = tempdf[tempdf.provider == p]
        amount_dict[b][p] = int(temp_p.shape[0])
        
    for s in tempdf.state.unique().tolist():
        temp_s = tempdf[tempdf.state == s]
        amount_dict[b][s] = int(temp_s.shape[0])
    


In [147]:
amount = pd.DataFrame.from_dict(amount_dict,orient='index')
amount = amount.fillna(0)
amount[['gocardless', 'stripe', 'paid_out', 'cancelled', 'pending_submission',
       'failed', 'transfer', 'charged_back', 'succeeded', 'confirmed',
       'submitted', 'unnax', 'paid']] = amount[['gocardless', 'stripe', 'paid_out', 'cancelled', 'pending_submission',
       'failed', 'transfer', 'charged_back', 'succeeded', 'confirmed',
       'submitted', 'unnax', 'paid']].astype(int)


In [148]:
amount.head()

,mean,std,max,min,count,payment_delay_mean,payment_delay_std,payment_delay_max,payment_delay_min,gocardless,...,cancelled,pending_submission,failed,transfer,charged_back,succeeded,confirmed,submitted,unnax,paid
-1,105.179019,143.071686,665.83,1.140,35,6 days,6 days,33 days,0 days,34,...,14,1,1,1,1,0,0,0,0,0
0,66.048277,92.050176,474.32,18.150,31,19 days,20 days,73 days,0 days,31,...,11,6,2,0,7,0,0,0,0,0
51936,115.689975,131.693994,406.56,16.456,8,5 days,3 days,8 days,0 days,8,...,2,0,0,0,0,0,0,0,0,0
52688,2.000000,0.000000,2.00,2.000,1,4 days,0 days,4 days,4 days,1,...,0,0,1,0,0,0,0,0,0,0
52712,2.420000,0.000000,2.42,2.420,1,0 days,0 days,0 days,0 days,0,...,0,0,0,1,0,0,0,0,0,0


In [143]:
amount.payment_delay_mean.dt.floor('1d')

-1         6 days
 0        19 days
 51936     5 days
 52688     4 days
 52712     0 days
 52731     0 days
 53300     8 days
 53305     0 days
 57613     7 days
 57674     0 days
 58757     0 days
 59306    10 days
 60101    89 days
 60104     8 days
 60445    13 days
 60886     0 days
 61006     0 days
 61868     0 days
 61908     0 days
 62188     0 days
 62202     0 days
 62417     0 days
 62436     0 days
 62534     0 days
 62634     9 days
 62677     7 days
 62687     7 days
 62695     1 days
 62929     7 days
 63169     0 days
            ...  
 99336     8 days
 99693    18 days
 99697     0 days
 99996     0 days
 100951   13 days
 101084    0 days
 101202    0 days
 101290    8 days
 101713    7 days
 101801    7 days
 102674   19 days
 102737    0 days
 102786    7 days
 103029    6 days
 103135    0 days
 103416    0 days
 103479    0 days
 103551    0 days
 103655    9 days
 103698    0 days
 104408    7 days
 104441    0 days
 104506    1 days
 104509    0 days
 104562   